## Part_3. Calculate Some Morphological Parameters of the Basin using GRASS GIS

In [125]:
import os
import sys
import subprocess
import shutil
import binascii
import tempfile

from grass_session import Session
import grass.script as gs
import grass.script.setup as gsetup

import numpy as np
import matplotlib
import wx
import math
import csv

In [2]:
os.chdir('/home/shirobakaidou/eagle/MET_Spatial_Python/grass')
os.getcwd()

'/home/shirobakaidou/eagle/MET_Spatial_Python/grass'

In [3]:
# Define GRASS Database
dbase = os.getcwd()
location = 'mylocation'

# Set GISBASE environment variable
gisbase = '/usr/lib/grass78'
os.environ['GISBASE'] = str(gisbase)

# Initialize
gsetup.init(os.environ['GISBASE'], dbase, location, 'PERMANENT')

print(gs.gisenv())

{'GISDBASE': '/home/shirobakaidou/eagle/MET_Spatial_Python/grass', 'LOCATION_NAME': 'mylocation', 'MAPSET': 'PERMANENT'}


In [4]:
# Create Location
gs.create_location(dbase, location)

In [20]:
# Create a new location, using a georeferenced file
#gs.run_command('g.proj', flags="c",
#               georef="/home/shirobakaidou/eagle/MET_Spatial_Python/data/hydroDEM_clip.tif",
#               location="rbasin_reproduce")

Location <rbasin_reproduce> created
You can switch to the new location by
`g.mapset mapset=PERMANENT location=rbasin_reproduce`


0

In [21]:
# Switch to the location_vietnam, mapset "PERMANENT"
gs.run_command('g.mapset', 
               #location="mylocation",
               location="rbasin_reproduce",
               mapset="PERMANENT")

Mapset switched. Your shell continues to use the history for the old mapset
You can switch the history by commands:
history -w; history -r
/home/shirobakaidou/eagle/MET_Spatial_Python/grass/rbasin_reproduce/PERMANENT/.bash_history;
HISTFILE=/home/shirobakaidou/eagle/MET_Spatial_Python/grass/rbasin_reproduce/PERMANENT/.bash_history


0

In [ ]:
# Install hydrology related GRASS Addons

#gs.run_command('g.extension', extension='r.basin', operation='add')
#gs.run_command('g.extension', extension='r.hypso', operation='add')
#gs.run_command('g.extension', extension='r.stream.basins', operation='add')
#gs.run_command('g.extension', extension='r.stream.distance', operation='add')
#gs.run_command('g.extension', extension='r.stream.order', operation='add')
#gs.run_command('g.extension', extension='r.stream.snap', operation='add')
#gs.run_command('g.extension', extension='r.stream.stats', operation='add')
#gs.run_command('g.extension', extension='r.width.funct', operation='add')
#gs.run_command('g.extension', extension='r.stream.channel', operation='add')
#gs.run_command('g.extension', extension='r.stream.segment', operation='add')
#gs.run_command('g.extension', extension='r.stream.slope', operation='add')

In [22]:
# Display CRS Info
gs.run_command('g.proj', flags="p")

-PROJ_INFO-------------------------------------------------
name       : WGS 84 / UTM zone 48N
datum      : wgs84
ellps      : wgs84
proj       : utm
zone       : 48
no_defs    : defined
-PROJ_EPSG-------------------------------------------------
epsg       : 32648
-PROJ_UNITS------------------------------------------------
unit       : meter
units      : meters
meters     : 1


0

In [24]:
# Read GeoTiFF as GRASS Raster (Import Raster)
gs.run_command('r.in.gdal', 
               #flags='e', # update the default region
               input='/home/shirobakaidou/eagle/MET_Spatial_Python/data/hydroDEM_clip.tif',
               output='r_elevation', 
               overwrite=True)

Importing raster map <r_elevation>...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100


0

In [25]:
# Set Region
gs.run_command('g.region', 
               flags="pa",
               raster='r_elevation')

projection: 1 (UTM)
zone:       48
datum:      wgs84
ellipsoid:  wgs84
north:      1998522.28534274
south:      1937087.40340192
west:       464222.85466713
east:       513265.29261131
nsres:      87.88967374
ewres:      87.88967374
rows:       699
cols:       558
cells:      390042


0

In [15]:
# Define Threshold:
# source: https://github.com/OSGeo/grass-addons/blob/master/grass7/raster/r.basin/r.basin.py
resolution = gs.region()['nsres']
th = 1000000 / (resolution**2)
gs.message( "threshold : %s" % th )

threshold : 129.45662942175616


In [58]:
# Define Output Location
outpath = os.path.join(os.path.split(os.getcwd())[0], 'output', 'location_reproduce_rbasin')

In [26]:
# Watershed SFD (single flow direction)
gs.run_command('r.watershed',
               flags='ams',
               elevation='r_elevation',
               accumulation='r_accumulation', # output accumulation
               drainage='r_drainage', # output flow direction
               convergence=5,
               overwrite=True)

SECTION 1 beginning: Initiating Variables. 4 sections total.
SECTION 1a: Mark masked and NULL cells
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100
SECTION 1b: Determining Offmap Flow.
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100
SECTION 2: A* Search.
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100
SECTION 3: Accumulating Surface Flow with SFD.
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  8

0

In [27]:
# Stream Extraction
gs.run_command('r.stream.extract', 
               elevation = 'r_elevation',
               accumulation = 'r_accumulation',
               threshold = th,
               #d8cut =  1000000000,
               mexp = 0,
               stream_rast = 'r_stream_e', # output unique stream ids
               direction = 'r_drainage_e',
               overwrite=True)

Loading input raster maps...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Initializing A* search...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
A* Search...
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100
Extracting streams...
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100
Thinning stream segments...
   0  12  25  37  50  62  75  87 100
Writing output raster maps...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  9

0

In [28]:
# Delineation of basin: Create outlet
# Create outlet
gs.write_command('v.in.ascii',
                 input='-',
                 output='v_outlet',
                 sep=",",
                 stdin="%s,9999" % ("490282.15625,1966838"), # <modify coordinates>
                 overwrite=True)
# The point is snapped to the nearest point which lies on the streamline
gs.run_command('r.stream.snap',
               input='v_outlet', 
               stream_rast='r_stream_e', # Input stream network
               output='v_outlet_snap', # output vector points map
               overwrite=True)
gs.run_command('v.to.rast',
               input='v_outlet_snap',
               output='r_outlet',
               use='cat',
               type='point',
               layer=1,
               value=1,
               overwrite=True)
# Delineates basins
gs.run_command('r.stream.basins',
               direction='r_drainage_e', # Input flow direction
               points='v_outlet_snap', # Input vector points
               basins='r_basin', # Output basin
               overwrite=True)
gs.message("Delineation of basin done")

Scanning input for column types...
Number of columns: 3
Number of rows: 1
Default driver / database set to:
driver: sqlite
database: $GISDBASE/$LOCATION_NAME/$MAPSET/sqlite/sqlite.db
Importing points...
   0 100
Populating table...
Building topology for vector map <v_outlet@PERMANENT>...
Registering primitives...
Reading raster map <r_stream_e>...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Building topology for vector map <v_outlet_snap@PERMANENT>...
Registering primitives...
Reading features...
 100
Writing raster map...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
v.to.rast complete.
Memory swap calculation (may take some time)...
Reading raster map <r_drainage_e>...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93

In [29]:
# Creation of Slope and Aspect maps
gs.run_command('r.slope.aspect',
               elevation='r_elevation',
               slope='r_slope',
               aspect = 'r_aspect',
               overwrite=True)

   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Aspect raster map <r_aspect> complete
Slope raster map <r_slope> complete


0

In [30]:
# Create Basin Mask (Vector)
gs.run_command('r.to.vect',
               input='r_basin',
               output='v_basin',
               type='area',
               flags='sv',
               overwrite=True)

Extracting areas...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Writing areas...
   0   4   8  12  16  20  24  28  32  36  40  44  48  52  56  60  64  68  72  76  80  84  88  92  96 100
Updating attributes...
   0 100
Building topology for vector map <v_basin@PERMANENT>...
Registering primitives...
Building areas...
   0 100
Attaching islands...
   0 100
Attaching centroids...
   0 100
r.to.vect complete.


0

In [31]:
# Add two columns to the table 'basin': Area and Perimeter
gs.run_command('v.db.addcolumn',
               map='v_basin',
               columns='area double precision') # name & type
gs.run_command('v.db.addcolumn',
               map='v_basin',
               columns='perimeter double precision')

0

In [32]:
# Populate Perimeter Column
gs.run_command('v.to.db',
               map='v_basin',
               type='line,boundary',
               layer=1,
               qlayer=1,
               option='perimeter',
               units='kilometers',
               columns='perimeter',
               overwrite=True)
gs.run_command('v.to.db',
               map='v_basin',
               type='line,boundary',
               layer=1,
               qlayer=1,
               option='area',
               units='kilometers',
               columns='area',
               #flags='c',
               overwrite=True)

Reading areas...
 100
Updating database...
   0 100
1 categories read from vector map (layer 1)
1 records selected from table (layer 1)
1 categories read from vector map exist in selection from table
1 records updated/inserted (layer 1)
Reading areas...
 100
Updating database...
   0 100
1 categories read from vector map (layer 1)
1 records selected from table (layer 1)
1 categories read from vector map exist in selection from table
1 records updated/inserted (layer 1)


0

In [65]:
# Export Selected Attributes
gs.run_command('v.db.select',
               map='v_basin',
               column='perimeter, area',
               file=os.path.join(outpath,'basin_attr'))

0

In [66]:
# Read Perimeter and Area from the text file
tmp = open(os.path.join(outpath,"basin_attr"), "r")
tmp = tmp.read()
tmp
#tmp.read().split('\n')

'perimeter|area\n111.389201|263.591175\n'

In [81]:
# Extract Perimeter and Area
perimeter_basin = float(tmp.split('\n')[1].split('|')[0])
area_basin = float(tmp.split('\n')[1].split('|')[1])
print(perimeter, area) # both km

111.389201 263.591175


In [52]:
# Create stream order maps: strahler, horton, hack, shreeve
gs.run_command('r.stream.order',
               stream_rast = 'r_stream_e',
               direction = 'r_drainage_e',
               strahler = 'r_strahler', # output Strahler order
               shreve = 'r_shreve', # output Shreve order
               horton = 'r_horton', # output Horton order
               hack = 'r_hack') # output Hack order

Memory swap calculation (may take some time)...
Reading raster map <r_stream_e>...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Reading raster map <r_drainage_e>...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Finding nodes...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Finding longest streams...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Calculating Strahler's stream order...
Calculating Hortons's stream order...
Calculating Shreve's stream magnitude, Scheidegger's consistent integer and
Drwal's streams hierarchy (old style)...
Calculating Hack's main streams and topological dimension...


0

In [54]:
# Distance to Outlet
gs.run_command('r.stream.distance',
               stream_rast = 'r_outlet',
               direction = 'r_drainage_e',
               flags = 'o',
               distance = 'r_distance')

Calculating segments in direction <DOWNSTREAM> (may take some time)...
Reading raster map <r_outlet>...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Reading raster map <r_drainage_e>...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Finding nodes...
Calculate downstream parameters...
   0 100
Writing raster map <r_distance>...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100


0

In [80]:
# Calc Mean Slope
baricenter_slope_baricenter = gs.read_command("r.volume",
                                              input='r_slope',
                                              clump='r_basin')
#mean_slope = baricenter_slope_baricenter[5]
#print(mean_slope)
#print(baricenter_slope_baricenter)
baricenter_slope_baricenter#.split('\n')[]

   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100


'\nVolume report on data from <r_slope> using clumps on <r_basin> raster map\n\nCategory   Average   Data   # Cells        Centroid             Total\nNumber     in clump  Total  in clump   Easting     Northing     Volume\n-----------------------------------------------------------------------------\n       1     11.38    385749   33888   482723.63  1985207.00    2979757814.20\n-----------------------------------------------------------------------------\n                                                Total Volume =  2979757814.20\n'

In [84]:
# Circularity Ratio
circularity_ratio = (4*math.pi*area_basin)/(perimeter_basin**2)
circularity_ratio

0.26696513826844187

In [87]:
# Main Channel
gs.mapcalc ("$r_mainchannel = if($r_hack==1,1,null())",
            r_hack = 'r_hack', # input
            r_mainchannel = 'r_mainchannel', # output
            overwrite=True) 

gs.run_command("r.thin", 
               input = 'r_mainchannel',
               output = 'r_mainchannel'+'_thin')

gs.run_command('r.to.vect',
               input='r_mainchannel'+'_thin',
               output='v_mainchannel',
               type='line',
               verbose=True)

Raster map <r_mainchannel> - 699 rows X 558 columns
Bounding box: l = 3, r = 519, t = 39, b = 621
Pass number 1
Deleted 0 pixels
Thinning completed successfully.
Output map 699 rows X 558 columns
Window 699 rows X 558 columns
Using native format
Extracting lines...
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
 100
Building topology for vector map <v_mainchannel@PERMANENT>...
Registering primitives...
8 primitives registered
529 vertices registered
Topology was built
Number of nodes: 16
Number of primitives: 8
Number of points: 0
Number of lines: 8
Number of boundaries: 0
Number of centroids: 0
Number of areas: 0
Number of isles: 0
r.to.vect complete.


0

In [88]:
# Get coordinates of the outlet (belonging to stream network)
gs.run_command('v.db.addtable', map='v_outlet_snap')

gs.run_command('v.db.addcolumn',
               map='v_outlet_snap',
               columns='x double precision, y double precision')

gs.run_command('v.to.db',
               map='v_outlet_snap',
               option='coor',
               col='x,y')

gs.run_command('v.out.ascii',
               input='v_outlet_snap',
               output=os.path.join(outpath,'outlet_coords.txt'),
               cats=1,
               format="point")

Reading features...
   0 100
Updating database...
   0 100
1 categories read from vector map (layer 1)
1 categories read from vector map don't exist in selection from table
1 records updated/inserted (layer 1)
Reading features...
   0 100
Updating database...
   0 100
1 categories read from vector map (layer 1)
1 records selected from table (layer 1)
1 categories read from vector map exist in selection from table
1 records updated/inserted (layer 1)
Fetching data...


0

In [93]:
outlet_coords = open(os.path.join(outpath,'outlet_coords.txt')).readline()
east_o, north_o, cat = outlet_coords.split('|')
print(east_o, north_o, cat)

490282.15625 1966838 1



In [101]:
# Basin Length
param_mainchannel = gs.read_command('v.what',
                                    map = 'v_mainchannel',
                                    coordinates='%s,%s' % (east_o, north_o),
                                    distance=5)
basin_length = float(param_mainchannel.split('\n')[7].split()[1])/1000 # km
#basin_length

75.032720702

In [102]:
# Elongation Ratio
# 1. Interpreted from Paper
elon_ratio1 = perimeter_basin/(math.pi*basin_length)

# 2. Based on r.basin
elon_ratio2 = (2*math.sqrt(area_basin/math.pi))/basin_length

print(elon_ratio1, elon_ratio2)

0.4725442921526407 0.24415734740038567


In [104]:
# Form Factor (Shape Factor)
form_factor = area_basin/(basin_length**2)
form_factor

0.046819791716406545

In [120]:
# Relative Relief (Hmax - Hmin)
height_attr = gs.read_command('r.info', 
                flags='r',
                map='r_elevation')
height_min = float(height_attr.strip().split('\n')[0].split('=')[-1])
height_max = float(height_attr.strip().split('\n')[1].split('=')[-1])
print(height_min, height_max)

relative_relief = height_max - height_min
print(relative_relief) # meter

165.0 1532.0
1367.0


In [121]:
# Relief Ratio
relief_ratio = (relative_relief/1000)/basin_length
relief_ratio

0.018218718276645972

In [122]:
# Dissection Index
dissection_index = relative_relief/height_max
dissection_index

0.8922976501305483

In [130]:
# Hypometric Curve
gs.run_command('r.hypso', 
               map = 'r_elevation',
               image=os.path.join(outpath,'hysometric_curve.png'),
               flags='a')

   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Gtk-Message: 16:05:38.628: Failed to load module "atk-bridge"
Gtk-Message: 16:05:38.630: Failed to load module "canberra-gtk-module"


Tot. cells 158241.0
Hypsometric | quantiles
823 | 0.025
755 | 0.05
693 | 0.1
582 | 0.25
436 | 0.5
250 | 0.75
196 | 0.9
176 | 0.975


Done!


0

In [129]:
# Write Indices to CSV File
csvfile = os.path.join(outpath, "output_indices.csv")
with open(csvfile, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Relative Relief (m)'] + [relative_relief])
    writer.writerow(['Relief Ratio'] + [relief_ratio])
    writer.writerow(['Dessection Index'] + [dissection_index])
    writer.writerow(['Form factor'] + [form_factor])
    writer.writerow(['Circularity Ratio'] + [circularity_ratio])
    writer.writerow(['Elongation Ratio'] + [elon_ratio1])
    writer.writerow(['Basin Area (km2)'] + [area_basin])
    writer.writerow(['Basin Perimeter (km)'] + [perimeter_basin])